In [2]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key  = ''

In [3]:
from langchain.document_loaders import PyPDFLoader

In [4]:
# The course will show the pip installs you would need to install packages on your own machine.
# These packages are already installed on this platform and should not be run again.
! pip install pypdf 

You should consider upgrading via the '/home/rj23/Desktop/LangChain/.venv/bin/python -m pip install --upgrade pip' command.


In [5]:
from langchain.document_loaders import PyPDFLoader

# List of file paths
file_paths = [
    "Insurance_Documents/about-our-insurance-services.pdf",
    "Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf",
    "Insurance_Documents/hsbc-life-privacy-notice-overview.pdf",
    "Insurance_Documents/life-cover-policy-document-and-summary-of-cover.pdf"
]

# Initialize a loader for each file and load pages
all_pages = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    all_pages.extend(pages)  # Collect all pages in a single list

# all_pages now contains pages from all documents


In [6]:
len(pages)

26

In [7]:
page=pages[1]

In [8]:
print(page.page_content)

2 Contents 
Policy Summary 3 
What is HSBC Life Cover?  3 
Can I apply for cover?  3 
How lo ng can the policy run for?  4 
How much cover can I choose?  4 
What type of cover can I apply for?  4 
How do I pay for the cover?  4 
What does the policy cover?  4 
Are there any other features included?  5 
Why might my policy not pay out?  5 
How do I make a claim?  6 
How do I cancel my policy?  6 
How do I make a complaint?  7 
Compensation 7 
Important Information about your policy  8 
Tax 8 
HSBC Life (UK) Limited  8 
Registered Office  9 
Your Informa tion 9 
Policy Document 10 
Introduction  10 
Policy Conditions  11 
Definitions  20 
An overview of h ow we collect and use your information  24 


In [9]:
page.metadata

{'source': 'Insurance_Documents/life-cover-policy-document-and-summary-of-cover.pdf',
 'page': 1}

In [10]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 150
)

In [11]:
splits = text_splitter.split_documents(all_pages)

In [12]:
len(splits)

194

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key='')

/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [15]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3= embedding.embed_query(sentence3)


In [16]:
import numpy as np

In [17]:
np.dot(embedding1, embedding2)

0.9631227500523621

In [18]:
np.dot(embedding1, embedding3)

0.7703257495981697

In [19]:
np.dot(embedding2, embedding3)

0.759162740110803

### Vectorstores

In [21]:
from langchain.vectorstores import Chroma

In [22]:
persist_directory = 'Insurance_Documents/chroma/'

In [23]:
!rm -rf ./Insurance_Documents/chroma 

In [24]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [25]:
print(vectordb._collection.count())

194


In [26]:
question = "What did they say about Policy Document?"

In [27]:
docs = vectordb.similarity_search(question,k=15)

In [28]:
len(docs)

15

In [29]:
print(docs[13].page_content)

which you can claim are given in the Policy Document under the Critical Illness Definitions heading  
Why might my policy not pay out?  
There are certain reasons your policy may not pay out in full or at all. The main reasons you need to be aware of are 
listed below.  
These are fully detailed in Conditions 4 and 5 of the Policy Conditions section of this booklet which you should read 
caref ully. 
General Exclusions  
There are some exclusions that you need to be aware of where we won’t pay a claim  
 You are first diagnosed with one of the specified critical illnesses or undergoes one of the specified surgical 
procedures within 90 days of the sta rt date of the Policy  
 You do not survive 10 days from either the date of diagnosis of one of the specified critical illnesses or the date of 
one of the specified surgical procedures (see Condition 1 of the Policy Conditions)


In [30]:
for doc in docs:
    print(doc.metadata)

{'page': 9, 'source': 'Insurance_Documents/life-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 10, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 10, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 9, 'source': 'Insurance_Documents/life-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 0, 'source': 'Insurance_Documents/life-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 0, 'source': 'Insurance_Documents/hsbc-life-privacy-notice-overview.pdf'}
{'page': 13, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 17, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 2, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 13, 'source': 'Insurance_Documents/life-cover-policy-document-and-su

In [31]:
vectordb.persist()

In [32]:
print(vectordb._collection.count())

194


In [33]:
docs_ss= vectordb.similarity_search("What is the policy document about?",k=3)

In [34]:
docs_ss[0].page_content[:100]

'10 Policy Document  \nIntroduction  \nImportant  \nThe terms and conditions of the Policy are set out i'

In [35]:
docs_ss[1].page_content[:100]

'\uf075 This Policy Document  \n\uf075 Policy Schedule(s)  \n\uf075 Any amendments, endorsements or specially agreed t'

In [36]:
question="What does the poicy cover for critical illness?"

In [37]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf"})


In [38]:
for d in docs:
    print(d.metadata)

{'page': 11, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 2, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 2, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}


In [39]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [40]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The policy the chunk is from, should be one of `Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf`, `Insurance_Docuuments/life-cover-policy-document-and-summary-of-cover.pdf`, or `about-our-insurance-services.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the policy the chunk is from",
        type="integer",
    ),
]

In [41]:

import lark

In [42]:

document_content_description = "A chunk of text from an insurance policy document."
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0, openai_api_key='')
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [43]:
docs = retriever.get_relevant_documents(question)

In [44]:
for d in docs:
    print(d.metadata)

{'page': 11, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 26, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 23, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}
{'page': 5, 'source': 'Insurance_Documents/critical-illness-cover-policy-document-and-summary-of-cover.pdf'}


COMPRESSION

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this.

In [45]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [46]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [48]:
# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct", openai_api_key='')
compressor = LLMChainExtractor.from_llm(llm)

In [49]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [50]:
question = "what does the policy cover for critical illness?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

- Critical illness cover (Life/Lives Insured)
- the benefit will become payable
- if after the start date of the Policy, the Life/Lives Insured is Diagnosed with one or more of the specified critical illnesses detailed in the Critical Illness Definitions section or has undergone one of the specified surgical procedures detailed in the Critical Illness Definitions and survives for a period of 10 days from either the date of Diagnosis or from the date of the specified surgical procedure.
----------------------------------------------------------------------------------------------------
Document 2:

- Your online Life Cover Policy can continue if you cancel your Critical Illness Policy.
- Your Critical Illness Policy must not exceed the Sum Insured or term on your online Life Policy.
- What is HSBC Critical Illness Cover?
- HSBC Critical Illness pays a lump sum is paid if you:
-  Suffer a critical Illness, or
-  Undergo a surgical procedure covered
- before the policy end 

In [51]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [52]:
question = "what does the policy cover for critical illness??"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

- Critical illness cover (Life/Lives Insured)
- the benefit will become payable
- the Life/Lives Insured is Diagnosed with one or more of the specified critical illnesses detailed in the Critical Illness Definitions section
- has undergone one of the specified surgical procedures detailed in the Critical Illness Definitions
- survives for a period of 10 days from either the date of Diagnosis or from the date of the specified surgical procedure
----------------------------------------------------------------------------------------------------
Document 2:

Critical Illness  – payment of the sum assured  
 If you are diagnosed with a Critical Illness or undergo a surgical procedure (see full list below)  
Children’s Critical illness  – payment of £25,000 or 50% of the Sum Insured (whichever is lower)  
 If any of your children (including natural , step or legally adopted) are diagnosed with one of the critical illnesses or 
undergo a surgical procedure (see full list below

In [54]:
print(vectordb._collection.count())

194


In [55]:
question = "What are the types of illnesses covered?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [57]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [59]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key='')

### Retrieval QA Chain

In [60]:
from langchain.chains import RetrievalQA

In [61]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever() # Use the vectorstore as the retriever
)

In [62]:
result = qa_chain({"query": question})

/home/rj23/Desktop/LangChain/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [63]:
result["result"]

'The types of illnesses covered under the HSBC Critical Illness policy include Aorta graft surgery for disease, Bacterial meningitis resulting in permanent symptoms, Benign Brain Tumour resulting in permanent symptoms or surgical removal, and Benign spinal cord tumour resulting in permanent symptoms.'

In [64]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [65]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [66]:
question = "What comes under serious illness?"

In [67]:
result = qa_chain({"query": question})

In [68]:
result["result"]

'Serious illnesses covered under the policy include heart valve replacement or repair, kidney failure requiring dialysis, loss of a hand or foot, loss of speech, aorta graft surgery for disease, and bacterial meningitis resulting in permanent symptoms. Thanks for asking!'

In [69]:
result["source_documents"][0]

Document(page_content='6 \uf075 Pulmonary artery surgery  – with surgery to divide the breastbone  \n\uf075 Removal of an eyeball  \n\uf075 Spinal stroke  – resulting in permanent symptoms  \n\uf075 Stroke  – resulting in permanent symptoms  \n\uf075 Terminal illness  \n\uf075 Third degree burns  – covering 20% of the body’s surface area  \n\uf075 Traumatic Brain injury  – resulting in permanent symptoms  \nThese headings are only a guide to what is covered by the Policy.  \nThese typically use medical terms to describe the illnesses but in some cases the cover may be limited. For example:  \n\uf075 Some types of cancer are not covered; and  \nTo make a claim for some critical illnesses you have to have permanent or persisting symptoms  \nFull details of these benefits are set out in the Policy Document section of this booklet.  \nThe full definitions of the specified critical illnesses and surgical procedures  covered and the circumstances in \nwhich you can claim are given in the Pol

## Retrieval QA Chain Types

In [70]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [71]:
result = qa_chain_mr({"query": question})

In [72]:
result["result"]

'Serious illnesses that may be covered under a policy typically include conditions such as heart attack, stroke, cancer, organ transplant, and other severe medical conditions. The specific list of serious illnesses covered can vary depending on the insurance policy. For full details on what is considered a serious illness under a specific policy, it is recommended to refer to the Policy Document section of the insurance booklet.'

In [75]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "ls__2de7c8d7ec664712a6123125c100f9c5" # replace dots with your api key

In [78]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

"Serious illnesses typically include conditions such as cancer, heart attack, stroke, organ failure, and other life-threatening diseases or injuries. It may also encompass chronic conditions that significantly impact a person's quality of life or require ongoing medical treatment. The specific list of serious illnesses covered will be outlined in the Policy Document under the Critical Illness Definitions heading. It is important to refer to the policy details for a comprehensive understanding of what qualifies as a serious illness for the purposes of making a claim."

In [77]:
from langsmith import Client

client = Client()

url = next(client.list_runs(project_name="default")).url
print(url)


https://smith.langchain.com/o/65d7e76c-a9dd-5c5b-b6af-2222bbd5a7ee/projects/p/ad5254ac-6c02-4131-8bab-ea947449ce3e/r/edbfe093-14cd-47e7-bb5c-9b0ed4f685ac?trace_id=92e144fa-b970-486c-96bc-1a3482a0728c&start_time=2024-04-21T22:29:20.973000


In [80]:
import panel as pn  # GUI
pn.extension()

ModuleNotFoundError: No module named 'panel'